In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score
from Constants import *
from Preprocessing import *
from Models import *
from Utils import *
pd.set_option('display.max_rows', 200)



In [14]:
Const.decisions

['Decision 1 (Induction Chemo) Y/N',
 'Decision 2 (CC / RT alone)',
 'Decision 3 Neck Dissection (Y/N)']

In [15]:
pd.read_csv('../data/digital_twin_ln_monograms.csv')

,id,L1A,L1B,L2A,L2B,L3,L4,L5A,L5B,L6,...,R1A,R1B,R2A,R2B,R3,R4,R5A,R5B,R6,RRPLN
0,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,10201,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
532,10202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
533,10203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
534,10204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [16]:
data = DTDataset(use_smote=True)
data.get_input_state(require=[Const.decisions[0]],step=2).loc[:,Const.decisions[1]].unique()
# data.processed_df#.shape, len(data.processed_df.index.unique())

/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


array([1, 0])

In [19]:
get_tt_split()[0]

/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


[5,
 6,
 8,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 21,
 23,
 24,
 26,
 27,
 28,
 32,
 33,
 37,
 38,
 39,
 40,
 41,
 42,
 48,
 49,
 50,
 51,
 53,
 55,
 56,
 57,
 60,
 64,
 65,
 67,
 69,
 71,
 74,
 75,
 78,
 79,
 80,
 81,
 82,
 87,
 88,
 91,
 94,
 96,
 99,
 103,
 109,
 116,
 119,
 120,
 121,
 125,
 148,
 150,
 153,
 178,
 181,
 183,
 185,
 186,
 188,
 191,
 192,
 193,
 196,
 197,
 198,
 200,
 201,
 203,
 204,
 205,
 206,
 207,
 210,
 212,
 213,
 214,
 216,
 218,
 219,
 220,
 221,
 222,
 223,
 225,
 226,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 237,
 238,
 239,
 240,
 241,
 243,
 244,
 246,
 247,
 248,
 249,
 251,
 252,
 253,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 265,
 266,
 269,
 270,
 273,
 275,
 276,
 277,
 278,
 280,
 281,
 282,
 283,
 285,
 289,
 2000,
 2002,
 2003,
 2004,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2016,
 2018,
 2021,
 2022,
 2023,
 2025,
 2027,
 2028,
 2030,
 2033,
 5000,
 5002,
 5004,
 5005,
 5006,
 5008,
 5009,
 5010,
 5011,
 5012,
 5

In [20]:
get_tt_split(resample_training=True)[0]

/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


array([10187,    40,   109, 10107, 10140, 10074,   226, 10090,   239,
         232,   266,   244,    39, 10064,   269,  5112, 10079, 10184,
       10132,  5067,  5023, 10052,   116, 10182,  2027,   153,    48,
         280,   240, 10018, 10082,  5010,  5095,  5026,  5004, 10115,
         234,  5066, 10073,    13,     8,   282,    96, 10127,  2012,
        5041,     8,    41,   196,  5113,  2000,  5027,    13, 10074,
          82,  5004,  5061,  5041,  2016,    14, 10006,   283,    56,
       10181,   206,    24,  5072,  5097, 10042, 10142, 10053,  2028,
       10096,    48,  2010, 10041, 10143,   275,   252,   261, 10042,
        5042, 10064, 10099, 10138,   238, 10022,   230,   270,     8,
         277,    48,   214,   204,  5029,  5043,   148,  5030,    99,
         281, 10075, 10054, 10127,  2013, 10099,   243,  5025,    41,
       10159, 10044, 10034, 10021, 10163,  2009,  2013,  5059,  5104,
         210,    38, 10096,  5019, 10025, 10004, 10121, 10099,    99,
          69,  5039,

In [21]:
def nllloss(ytrue,ypred):
    #nll loss with argmax added in
    loss = torch.nn.NLLLoss()
    return loss(ypred,ytrue.argmax(axis=1))

def state_loss(ytrue,ypred,weights=[1,1,1,1]):
    pd_loss = nllloss(ytrue[0],ypred[0])*weights[0]
    nd_loss = nllloss(ytrue[1],ypred[1])*weights[1]
    mod_loss = nllloss(ytrue[2],ypred[2])*weights[2]
    loss = pd_loss + nd_loss + mod_loss
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
#     nloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    for i in range(ndlt):
        dlt_loss = bce(dlt_pred[:,i].view(-1),dlt_true[:,i].view(-1))
        loss += dlt_loss*weights[3]/ndlt
    return loss

def outcome_loss(ytrue,ypred,weights=[1,1,1]):
    loss = 0
    nloss = torch.nn.BCELoss()
    for i in range(len(weights)):
        iloss = nloss(ypred[:,i],ytrue[i])*weights[i]
        loss += iloss
    return loss

def mc_metrics(yt,yp,numpy=False,is_dlt=False):
    if not numpy:
        yt = yt .cpu().detach().numpy()
        yp = yp.cpu().detach().numpy()
    #dlt prediction (binary)
    if is_dlt:
        acc = accuracy_score(yt,yp>.5)
        if yt.sum() > 1:
            auc = roc_auc_score(yt,yp)
        else:
            auc=-1
        error = np.mean((yt-yp)**2)
        return {'accuracy': acc, 'mse': error, 'auc': auc}
    #this is a catch for when I se the dlt prediction format (encoded integer ordinal, predict as a categorical and take the argmax)
    elif yt.ndim > 1:
        try:
            bacc = balanced_accuracy_score(yt.argmax(axis=1),yp.argmax(axis=1))
        except:
            bacc = -1
        try:
            roc_micro = roc_auc_score(yt,yp,average='micro')
        except:
            roc_micro=-1
        try:
            roc_macro = roc_auc_score(yt,yp,average='macro')
        except:
            roc_macro = -1
        return {'accuracy': bacc, 'roc_micro': roc_micro,'roc_macro': roc_macro}
    #outcomes (binary)
    else:
        if yp.ndim > 1:
            yp = yp.argmax(axis=1)
        try:
            bacc = accuracy_score(yt,yp)
        except:
            bacc = -1
        try:
            roc = roc_auc_score(yt,yp)
        except:
            roc = -1
        error = np.mean((yt-yp)**2)
        return {'accuracy': bacc, 'mse': error, 'auc': roc}

def state_metrics(ytrue,ypred,numpy=False):
    pd_metrics = mc_metrics(ytrue[0],ypred[0],numpy=numpy)
    nd_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    mod_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    
    dlt_metrics = []
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
    nloss = torch.nn.NLLLoss()
    for i in range(ndlt):
        dm = mc_metrics(dlt_true[:,i],dlt_pred[:,i].view(-1),is_dlt=True)
        dlt_metrics.append(dm)
    dlt_acc =[d['accuracy'] for d in dlt_metrics]
    dlt_error = [d['mse'] for d in dlt_metrics]
    dlt_auc = [d['auc'] for d in dlt_metrics]
    
    acc_mean = np.mean([a for a in dlt_acc if a >= 0])
    auc_mean = np.mean([a for a in dlt_auc if a >= 0])
    results = {'pd': pd_metrics,'nd': nd_metrics,'mod': mod_metrics,
               'dlts': {'accuracy': dlt_acc,'accuracy_mean': acc_mean,'auc': dlt_auc,'auc_mean': auc_mean}
              }
    return results

def outcome_metrics(ytrue,ypred,numpy=False):
    res = {}
    for i, outcome in enumerate(Const.outcomes):
        metrics = mc_metrics(ytrue[i],ypred[:,i])
        res[outcome] = metrics
    return res

def transition_sample(state):
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split()
    
    dataset = DTDataset()
    
    #only train on people with  IC for state 1 since other people can't have any outcomes otherwise
    require = None
    if state == 1:
        require = Const.decisions[0] #we don't expect a state update if there is no treatment
        valid_ids = dataset.get_input_state(require=require).index.values
        train_ids = [t for t in train_ids if t in valid_ids]
        test_ids = [t for t in test_ids if t in valid_ids]
    xtrain = dataset.get_input_state(step=state,ids=train_ids,require=require)
    xtest = dataset.get_input_state(step=state,ids=test_ids,require=require)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids,require=require)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids,require=require)

    xtrain = df_to_torch(xtrain)
    xtest = df_to_torch(xtest)
    ytrain = [df_to_torch(t) for t in ytrain]
    ytest= [df_to_torch(t) for t in ytest]
    return xtrain,xtest,ytrain,ytest

In [23]:
def train_state(model=None,
                model_args={},
                state=1,
                split=.7,
                lr=.0001,
                epochs=1000,
                patience=10,
                use_attention=False,
                weights=None,
                save_path='../data/models/',
                use_default_split=True,
                use_bagging_split=False,
                resample_training=False,#use bootstraping on training data after splitting
                n_validation_trainsteps=2,
                verbose=True,
                use_smote=True,
                file_suffix=''):
    
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)

    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()
    
    #only train on people with  IC for state 1 since other people can't have any outcomes otherwise
    require = None
    if state == 1:
        require = Const.decisions[0] #we don't expect a state update if there is no treatment
        valid_ids = dataset.get_input_state(require=require).index.values
        train_ids = [t for t in train_ids if t in valid_ids]
        test_ids = [t for t in test_ids if t in valid_ids]
    xtrain = dataset.get_input_state(step=state,ids=train_ids,require=require)
    xtest = dataset.get_input_state(step=state,ids=test_ids,require=require)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids,require=require)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids,require=require)
    

    if not use_attention:
        model_args = {k:v for k,v in model_args.items() if 'attention' not in k and 'embed_size' not in k}
    if state < 3:
        if model is None:
            if use_attention:
                model = OutcomeAttentionSimulator(xtrain.shape[1],state=state,**model_args)
            else:
                model = OutcomeSimulator(xtrain.shape[1],state=state,**model_args)
        lfunc = state_loss
        if weights is None:
            weights = [1,1,1,1]
    else:
        if model is None:
            if use_attention:
                model = EndpointAttentionSimulator(xtrain.shape[1],**model_args)
            else:
                model = EndpointSimulator(xtrain.shape[1],**model_args)
        if weights is None:
            weights = [1,1,1,1]
        lfunc = outcome_loss
        
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.set_device(device)
    
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    save_file = save_path + 'model_' + model.identifier + '_split' + str(split) + '_resample' + str(resample_training) +  '_hash' + hashcode + file_suffix + '.tar'
    xtrain = df_to_torch(xtrain).to(device)
    xtest = df_to_torch(xtest).to(device)
    ytrain = [df_to_torch(t).to(device) for t in ytrain]
    ytest= [df_to_torch(t).to(device) for t in ytest]
    
    model.fit_normalizer(xtrain)
#     normalize = lambda x: (x - xtrain.mean(axis=0)+.01)/(xtrain.std(axis=0)+.01)
#     unnormalize = lambda x: (x * (xtrain.std(axis=0) +.01)) + xtrain.mean(axis=0) - .01
    
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    best_val_loss = 1000000000000000000000000000
    best_loss_metrics = {}
    last_epoch = False
    for epoch in range(epochs):
        
        model.train(True)
        optimizer.zero_grad()
        
        xtrain_sample = xtrain#[torch.randint(len(xtrain),(len(xtrain),) )]
        ypred = model(xtrain_sample)
        loss = lfunc(ytrain,ypred,weights=weights)

        loss.backward()
        optimizer.step()
        if verbose:
            print('epoch',epoch,'train loss',loss.item())
        
        model.eval()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        if state < 3:
            val_metrics = state_metrics(ytest,yval)
        else:
            val_metrics = outcome_metrics(ytest,yval)
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_loss_metrics = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if verbose:
            print('val loss',val_loss.item())
            print('______________')
        if steps_since_improvement > patience:
            break
    print('best loss',best_val_loss,best_loss_metrics)
    model.load_state_dict(torch.load(save_file))
    
    #train one step on validation data
    for i in range(n_validation_trainsteps):
        model.train()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        val_loss.backward()
        optimizer.step()
        torch.save(model.state_dict(),save_file)
    
    model.eval()
    return model,  best_val_loss, best_loss_metrics

In [24]:
def train_model_ensemble(model_arg_list,state,
                         n_bags=10,
                         smote_options=[True,False],
                         smote_baseline_options=[True,False],
                         weight_options=[None],
                         weight_baseline_options=[None],
                         **kwargs):
    resampled_models =[]
    base_models = []
    base_metrics = []
    resample_metrics = []
    base_losses = []
    resample_losses = []
    n_errors = 0
    for margs in model_arg_list:
        for s in smote_baseline_options:
            for w in weight_baseline_options:
                [base_model,blosses,bmetrics] = train_state(model_args=margs,state=state,
                                         resample_training=False,
                                         use_smote=smote_options,
                                         weights=w,
                                         verbose=False,**kwargs)
            base_models.append(base_model)
            base_metrics.append(bmetrics)
            base_losses.append(blosses)
        for n in range(n_bags):
            for s in smote_options:
                for w in weight_options:
                    #this can fail if I resample a bad distribution with no minority classes
                    done = False
                    while not done:
                        try:
                            [model,loss,metrics] = train_state(model_args=margs,
                                                state=state,resample_training=True,
                                                use_smote=s,
                                                weights=w,
                                                verbose=False,**kwargs)
                            resampled_models.append(model)
                            resample_metrics.append(metrics)
                            resample_losses.append(loss)
                            done=True
                            print('model_done',len(base_models)+len(resampled_models))
                        except Exception as e:
                            print('error training model')
                            print(e)
                            n_errors += 1
    print('done with',n_errors,'errors')
    return base_models,resampled_models

In [30]:
test = torch.FloatTensor([.4])
test.item()

0.4000000059604645

In [31]:
# t1_arglist = [
#      {'hidden_layers': [400,400], 'dropout': 0.9, 'input_dropout': 0.1},
#     {'hidden_layers': [400], 'dropout': .7, 'input_dropout': .2}
# ]
# m1_list, m1_ensemble = train_model_ensemble(t1_arglist,1,
#                                             n_bags=5,
#                                             weight_options=[[1,1,1,10],[2,2,0.1,1]],
#                                             weight_baseline_options=[[1,1,1,10],[2,2,0.1,1]],
#                                            )
from Models import *
emodel1 = TransitionEnsemble(m1_list,m1_ensemble)
test2 = emodel1(transition_sample(1)[0][0].view(1,-1).to(emodel1.get_device()))
test2

/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


{'predictions': [tensor([[0.1824, 0.7747, 0.0429]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.1170, 0.8334, 0.0496]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.6876, 0.0529, 0.0476, 0.0533, 0.0470, 0.0662, 0.0454]],
         device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.1417, 0.1310, 0.0811, 0.0788, 0.1119, 0.0764, 0.0651, 0.1352]],
         device='cuda:0', grad_fn=<MeanBackward1>)],
 '5%': [tensor([[0.0920, 0.3596, 0.0234]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.0242, 0.5809, 0.0191]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.1872, 0.0084, 0.0071, 0.0090, 0.0059, 0.0107, 0.0042]],
         device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.0229, 0.0175, 0.0043, 0.0049, 0.0324, 0.0040, 0.0068, 0.0159]],
         device='cuda:0', grad_fn=<ViewBackward0>)],
 '95%': [tensor([[0.4612, 0.8766, 0.2054]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.2185, 0.9571, 0.2028]], device='cuda:0', grad_fn=<ViewBac

In [32]:
t2_arglist = [
     {'hidden_layers': [400,400], 'dropout': 0.9, 'input_dropout': 0.1},
    {'hidden_layers': [400], 'dropout': .7, 'input_dropout': .2}
]
m2_list, m2_ensemble = train_model_ensemble(t2_arglist,2,
                                            weight_options=[[1,1,1,10],[2,2,0.1,1]],
                                            weight_baseline_options=[[1,1,1,10],[2,2,0.1,1]],
                                            n_bags=5)
emodel2 = TransitionEnsemble(m2_list,m2_ensemble)
emodel2(transition_sample(2)[0][0].view(1,-1))

/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


here


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred cont

best loss 3.2491872310638428 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9466635059855399, 'roc_macro': -1}, 'nd': {'accuracy': 0.35897943037974683, 'roc_micro': 0.7228892471307236, 'roc_macro': 0.4681364683270171}, 'mod': {'accuracy': 0.35897943037974683, 'roc_micro': 0.7228892471307236, 'roc_macro': 0.4681364683270171}, 'dlts': {'accuracy': [0.9657534246575342, 0.952054794520548, 1.0, 1.0, 0.9657534246575342, 1.0, 0.9794520547945206, 0.9178082191780822], 'accuracy_mean': 0.9726027397260275, 'auc': [0.5886524822695036, 0.7153134635149023, -1, -1, 0.526241134751773, -1, 0.5710955710955711, 0.6623134328358209], 'auc_mean': 0.6127232168935143}}


/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


...

best loss 3.5613417625427246 {'pd': {'accuracy': 0.3361513296407865, 'roc_micro': 0.9435044181237074, 'roc_macro': 0.6444895146837865}, 'nd': {'accuracy': 0.36322213449138047, 'roc_micro': 0.7476821983273596, 'roc_macro': 0.5761739801698239}, 'mod': {'accuracy': 0.36322213449138047, 'roc_micro': 0.7476821983273596, 'roc_macro': 0.5761739801698239}, 'dlts': {'accuracy': [0.9761092150170648, 0.9385665529010239, 1.0, 1.0, 0.962457337883959, 0.9931740614334471, 0.9761092150170648, 0.931740614334471], 'accuracy_mean': 0.9722696245733788, 'auc': [0.5749250749250749, 0.5042424242424242, -1, -1, 0.6070277240490007, 0.34707903780068733, 0.5424575424575424, 0.6553113553113553], 'auc_mean': 0.5385071931310141}}
model_done 43


/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning

best loss 2.9335005283355713 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9221730629433735, 'roc_macro': -1}, 'nd': {'accuracy': 0.33163920119102624, 'roc_micro': 0.7419054259634889, 'roc_macro': 0.5965891077654472}, 'mod': {'accuracy': 0.33163920119102624, 'roc_micro': 0.7419054259634889, 'roc_macro': 0.5965891077654472}, 'dlts': {'accuracy': [0.9753521126760564, 0.9436619718309859, 1.0, 1.0, 0.9612676056338029, 0.9823943661971831, 0.9753521126760564, 0.9366197183098591], 'accuracy_mean': 0.971830985915493, 'auc': [0.739556472408458, 0.6870335820895522, -1, -1, 0.49950049950049946, 0.8272401433691756, 0.6044352759154203, 0.5632832080200502], 'auc_mean': 0.6535081968838593}}
model_done 44
done with 5 errors


/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


{'predictions': [tensor([[0.7630, 0.2035, 0.0335]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.3986, 0.5348, 0.0666]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.2485, 0.5783, 0.0987, 0.0746]], device='cuda:0',
         grad_fn=<MeanBackward1>),
  tensor([[0.0633, 0.0881, 0.0520, 0.0529, 0.0741, 0.0682, 0.0838, 0.0882]],
         device='cuda:0', grad_fn=<MeanBackward1>)],
 '5%': [tensor([[0.6609, 0.0815, 0.0183]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.3187, 0.3215, 0.0241]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.0193, 0.4435, 0.0342, 0.0113]], device='cuda:0',
         grad_fn=<ViewBackward0>),
  tensor([[0.0089, 0.0170, 0.0029, 0.0031, 0.0207, 0.0094, 0.0119, 0.0158]],
         device='cuda:0', grad_fn=<ViewBackward0>)],
 '95%': [tensor([[0.8865, 0.2883, 0.0761]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.6356, 0.6482, 0.0816]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.2933, 0.9299, 0.2027, 0.1

In [36]:
t3_arglist = [
     {'hidden_layers': [50], 'dropout': 0.5, 'input_dropout': 0},
]
m3_list, m3_ensemble = train_model_ensemble(t3_arglist,3,n_bags=5,
                                            weight_options=[[1,1,1,1],[1,2,2,1]],
                                            weight_baseline_options=[[1,1,1,1],[1,2,2,1]],
                                           )
emodel3 = EndpointEnsemble(m3_list,m3_ensemble)
emodel3(transition_sample(3)[0][0].view(1,-1))

/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


{'predictions': tensor([[0.7222, 0.3142, 0.3264, 0.7353]], device='cuda:0',
        grad_fn=<MeanBackward1>),
 '5%': tensor([[0.6279, 0.2141, 0.2462, 0.7228]], device='cuda:0',
        grad_fn=<ViewBackward0>),
 '95%': tensor([[0.8327, 0.5555, 0.5261, 0.9048]], device='cuda:0',
        grad_fn=<ViewBackward0>)}

In [ ]:
emodel3

In [ ]:
def gridsearch_attention_transition_models(state=1,attentions=[False]):
    hiddenlayers = [[10],[50],[100],[500],[1000],[10,10],[50,50],[100,100],[500,500],[1000,1000]]
    model_arglist = [{'hidden_layers': h} for h in hiddenlayers]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    k = 0
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for use_smote in [True,False]:
            for dropout in [0,.1,.25,.5,.9]:
                args['dropout'] = dropout
                for input_dropout in [0,.1,.25,.5]:
                    args['input_dropout'] = input_dropout
                    model,m_loss,m_metrics = train_state(
                        model_args=args,
                        state=state,
                        use_smote=use_smote,
                        use_attention=False,
                        verbose=False)
                    print('done',k,m_loss)
                    k+=1
                    if m_loss < best_loss:
                        best_loss = m_loss
                        best_metrics  = m_metrics
                        best_model = model
                        best_args = {k:v for k,v in args.items()}
                        best_args['smote'] = use_smote
                        print('_++++++++++New Best++++____')
                        print(best_loss)
                        print(best_metrics)
                        print(best_args)
                        print(use_smote)
                        print('___________')
                        print('++++++++')
                        print()
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model
# model1 = gridsearch_attention_transition_models(1)
# model1

In [ ]:
# model2 = gridsearch_attention_transition_models(2)
# model2

In [ ]:
# model3 = gridsearch_attention_transition_models(3)
# model3

In [ ]:
# def load_trained_models():
#     files = Const.tuned_transition_models
#     decision_file = Const.tuned_decision_model
#     [model1,model2,model3] = [torch.load(file) for file in files]
#     decision_model = torch.load(decision_file)
#     return decision_model, model1,model2,model3
# _, model1, model2, model3 =load_trained_models()

In [44]:

def shuffle_col(v,col=None):
    if col is None:
        col = np.random.choice([i for i in range(v.shape[1])])
    idx = torch.randperm(v.shape[0])
    vv = torch.clone(v)
    vv[:,col] = vv[idx,col]
    return vv
    
def train_decision_model(
    tmodel1,
    tmodel2,
    tmodel3,
    use_default_split=True,
    use_bagging_split=False,
    lr=.0001,
    epochs=10000,
    patience=100,
    weights=[1,1,0,0], #realtive weight of survival, feeding tube, and aspiration
    imitation_weights=[1,1,1],#weights of imitation decisions, because ic overtrains too quickly
    imitation_weight=0,
    shufflecol_chance = 0.1,
    reward_weight=1,
    split=.7,
    resample_training=False,
    save_path='../data/models/',
    file_suffix='',
    use_attention=True,
    verbose=True,
    threshold_decisions=False,#convert decisiosn to binary in simulation, usually breaks it
    use_smote=False,
    validate_with_memory=True,
    **model_kwargs,
):
    tmodel1.eval()
    tmodel2.eval()
    tmodel3.eval()

    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)
    true_ids = train_ids + test_ids #for saving memory without upsampling
    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()
    data = dataset.processed_df.copy()
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        #this should have an ic condition but we don't use it anumore anywa
        return res
        
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline')
    
    def formatdf(d,dids=train_ids):
        d = df_to_torch(d.loc[dids]).to(model.get_device())
        return d
    
    def makegrad(v):
        if not v.requires_grad:
            v.requires_grad=True
        return v
    
    if use_attention:
        model = DecisionAttentionModel(baseline.shape[1],**model_kwargs)
    else:
        model_kwargs = {k:v for k,v in model_kwargs.items() if 'attention' not in k and 'embed' not in k}
        model = DecisionModel(baseline.shape[1],**model_kwargs)

    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    
    save_file = save_path + 'model_' + model.identifier +'_hash' + hashcode + file_suffix + '.tar'
    model.fit_normalizer(df_to_torch(baseline.loc[train_ids]))
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)

    def outcome_loss(ypred):
        #convert survival to death
        l = torch.mul(torch.mean(-1*(ypred[:,0] - 1)),weights[0])
        for i,weight in enumerate(weights[1:]):
            #weights with negative values will invert the outcome so e.g. Regional control becomes no regional control
            #so the penaly is correct
            if weight > 0:
                newloss = torch.mean(ypred[:,i])*weight
            else:
                newloss = torch.mul(torch.mean(-1*(ypred[:,0] - 1)),-weights[i])
            l = torch.add(l,torch.mul(newloss,weight))
        return l
    
    mse = torch.nn.MSELoss()
    nllloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    
    def compare_decisions(d1,d2,d3,ids):
#         ypred = np.concatenate([dd.cpu().detach().numpy().reshape(-1,1) for dd in [d1,d2,d3]],axis=1)
        ytrue = df_to_torch(outcomedf.loc[ids])
        dloss = bce(d1.view(-1),ytrue[:,0])
        dloss += bce(d2.view(-1),ytrue[:,1])
        dloss += bce(d3,view(-1),ytrue[:,2])
        return dloss
        
    def remove_decisions(df):
        cols = [c for c in df.columns if c not in Const.decisions ]
        ddf = df[cols]
        return ddf
    
    makeinput = lambda step,dids: df_to_torch(remove_decisions(dataset.get_input_state(step=step,ids=dids)))
    threshold = lambda x: torch.gt(x,.5).type(torch.FloatTensor)

    
    #save the inputs from the whole dataset for future reference
    if use_attention:
        full_data = []
        for mstep in [0,1,2]:
            full_data_step = [baseline, get_dlt(min(mstep,1)),
                         get_dlt(mstep),get_pd(mstep),get_nd(mstep),get_cc(mstep),get_mod(mstep)]
            full_data_step = torch.cat([formatdf(fd,true_ids) for fd in full_data_step],axis=1)
            full_data.append(full_data_step)
        full_data = torch.stack(full_data)
        model.save_memory(full_data)
        print(full_data.shape)
    def step(train=True):
        if train:
            model.train(True)
#             tmodel1.train(True)
#             tmodel2.train(True)
#             tmodel3.train(True)
            optimizer.zero_grad()
            ids = train_ids
        else:
            ids = test_ids
            model.eval()
#             tmodel1.eval()
#             tmodel2.eval()
#             tmodel3.eval()
            
            
        ytrain = df_to_torch(outcomedf.loc[ids])
        #imitation losses and decision 1
        xxtrained = [baseline, get_dlt(0),get_dlt(0),get_pd(0),get_nd(0),get_cc(0),get_mod(0)]
        xxtrain = [formatdf(xx,ids) for xx in xxtrained]
        
        use_memory = (not train) and validate_with_memory
        if use_attention:
            o1 = model(torch.cat(xxtrain,axis=1),position=0,use_saved_memory = use_memory)
        else:
            o1 = model(torch.cat(xxtrain,axis=1),position=0)
#         print('o1',o1.grad)
        decision1_imitation = o1[:,3]
        decision1 = o1[:,0]
        if threshold_decisions:
            decision1 = threshold(decision1)
#         imitation_loss1 = bce(threshold(decision1_imitation),ytrain[:,0])
        imitation_loss1 = bce(decision1_imitation,ytrain[:,0])
        imitation_loss1 = torch.mul(imitation_loss1,imitation_weights[0])
        
        x1_imitation = [baseline, get_dlt(1),get_dlt(0),get_pd(1),get_nd(1),get_cc(1),get_mod(1)]
        x1_imitation = [formatdf(xx1,ids) for xx1 in x1_imitation]
        
        if use_attention:
            decision2_imitation = model(torch.cat(x1_imitation,axis=1),position=1,use_saved_memory = use_memory)[:,4]
        else:
            decision2_imitation = model(torch.cat(x1_imitation,axis=1),position=1)[:,4]
            
        imitation_loss2 =  bce(decision2_imitation,ytrain[:,1])
        imitation_loss2 = torch.mul(imitation_loss2,imitation_weights[1])
        
        x2_imitation = [baseline, get_dlt(1),get_dlt(2),get_pd(2),get_nd(2),get_cc(2),get_mod(2)]
        
        x2_imitation = [formatdf(xx2,ids) for xx2 in x2_imitation]
        
        if use_attention:
            decision3_imitation = model(torch.cat(x2_imitation,axis=1),position=2,use_saved_memory = use_memory)[:,5]
        else:
            decision3_imitation = model(torch.cat(x2_imitation,axis=1),position=2)[:,5]
        imitation_loss3 = bce(decision3_imitation,ytrain[:,2])
        imitation_loss3 = torch.mul(imitation_loss3,imitation_weights[2])
        
        #reward decisions
        xx1 = makeinput(1,ids)
        xx2 = makeinput(2,ids)
        xx3 = makeinput(3,ids)

        xx1 = makegrad(xx1)
        xx2 = makegrad(xx2)
        xx3  = makegrad(xx3)
        baseline_train_base = formatdf(baseline,ids)
            
        baseline_train = torch.clone(baseline_train_base)
#         if train and shufflecol_chance > 0.0001:
#             for col in range(baseline_train_base.shape[1]): 
#                 if np.random.random() < shufflecol_chance:
#                     baseline_train = shuffle_col(baseline_train,col)
                    
        
        xi1 = torch.cat([xx1,decision1.view(-1,1)],axis=1)
        [ypd1, ynd1, ymod, ydlt1] = tmodel1(xi1)['predictions']
        
#         print('t1',[v.grad for v in [ypd1,ynd1,ymod,ydlt1]])
        #this outputs log likelihoods (except for dlts) -> convert to probability
        #with ensemble stuff this is obselete
#         ypd1 = torch.exp(ypd1)
#         ynd1 = torch.exp(ynd1)
        d1_thresh = torch.gt(decision1.view(-1,1),.5).to(ypd1.device)
        d1_scale = torch.cat([d1_thresh,d1_thresh,torch.ones(d1_thresh.view(-1,1).shape).to(ypd1.device)],dim=1)
        ypd1= torch.mul(ypd1,d1_scale)
        ynd1= torch.mul(ynd1,d1_scale)
#         ymod = torch.exp(ymod)
        
        x1 = [baseline_train,ydlt1,formatdf(get_dlt(0),ids),ypd1,ynd1,formatdf(get_cc(1),ids),ymod]
        x1=[xx1.to(model.get_device()) for xx1 in x1]
        if use_attention:
            decision2 = model(torch.cat(x1,axis=1),position=1,use_saved_memory = use_memory)[:,1] 
        else:
            decision2 = model(torch.cat(x1,axis=1),position=1)[:,1] 
        if threshold_decisions:
            decision2 = threshold(decision2)
            
        xi2 = torch.cat([xx2,decision1.view(-1,1),decision2.view(-1,1)],axis=1)
        [ypd2,ynd2,ycc,ydlt2] = tmodel2(xi2)['predictions']
#         print('t2',[v.grad  for v in [ypd2,ynd2,ycc,ydlt2]])
#         ypd2 = torch.exp(ypd2)
#         ynd2 = torch.exp(ynd2)
        
#         ycc = torch.exp(ycc)
        x2 = [baseline_train,ydlt1,ydlt2,ypd2,ynd2,ycc,ymod]
        x2 = [xx2.to(model.get_device()) for xx2 in x2]
        if use_attention:
            decision3 = model(torch.cat(x2,axis=1),position=2,use_saved_memory = use_memory)[:,2]
        else:
            decision3 = model(torch.cat(x2,axis=1),position=2)[:,2]
        if threshold_decisions:
            decision3 = threshold(decision3)
            

        xi3 = torch.cat([xx3,decision1.view(-1,1),decision2.view(-1,1),decision3.view(-1,1)],axis=1)
        outcomes = tmodel3(xi3)['predictions']
#         print('t3',outcomes.grad)
        if not train and verbose:
            print(torch.mean(outcomes,dim=0))
        reward_loss = outcome_loss(outcomes)

        loss = torch.add(imitation_loss1,imitation_loss2)
        loss = torch.add(loss,imitation_loss3)
        loss = torch.mul(loss,imitation_weight/3)
        loss = torch.add(loss,torch.mul(reward_loss,reward_weight))
        losses = [imitation_loss1+imitation_loss2+imitation_loss3,reward_loss]
        
        if not train:
            distributions = [torch.mean(torch.gt(d,.5).type(torch.FloatTensor)) for d in [decision1,decision2,decision3]]
            if verbose:
                print('decisions',distributions)
#         print('decitions', 'train',train,decision1.grad,decision2.grad,decision1_imitation.grad)
        if train:
#             reward_loss.backward()
            loss.backward()
            optimizer.step()
            return losses
        else:
            scores = []
            for i,decision in enumerate([decision1_imitation,decision2_imitation,decision3_imitation]):
                dec = decision.cpu().detach().numpy()
                dec0 = (dec > .5).astype(int)
                out = ytrain[:,i].cpu().detach().numpy()
                acc = accuracy_score(out,dec > .5)
                auc = roc_auc_score(out,dec)
                scores.append({'decision': i,'accuracy': acc,'auc': auc})
            return losses, scores, distributions
        
    best_val_loss = torch.tensor(1000000000.0)
    steps_since_improvement = 0
    best_val_score = {}
    
    for epoch in range(epochs):
        losses = step(True)
        val_losses,val_metrics,val_distributions = step(False)
        #[1] is rl loss, use the second one as a tiebreaker
#         vl = val_losses[1] + val_losses[0]/10
        aucsum = np.sum([d['auc'] for d in val_metrics])
        vl = val_losses[1] - aucsum
        if verbose:
            print('______epoch',str(epoch),'_____')
            print('val reward',val_losses[1].item())
            print('imitation reward', val_losses[0].item())
            print('distributions',val_distributions)
            print(val_metrics)
        if vl < best_val_loss:
            best_val_loss = vl
            best_val_score = val_metrics
            best_val_distributions = val_distributions
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if steps_since_improvement > patience:
            break
    print('++++++++++Final+++++++++++')
    print('best',best_val_loss)
    print(best_val_score)
    model.load_state_dict(torch.load(save_file))
    model.eval()
    return model, best_val_score, best_val_loss, best_val_distributions

from Models import *
args = {
    'hidden_layers': [50,50], 
    'attention_heads': [1,1], 
    'embed_size': 120, 
    'dropout': 0.5, 
    'input_dropout': 0, 
    'shufflecol_chance':  .5,
}

#1.8424
decision_model, _, _, _ = train_decision_model(
    emodel1,emodel2,emodel3,
    lr=.0001,
    use_attention=True,
    imitation_weight=1,
    reward_weight=1,
    weights=[1,1,1,1],
    threshold_decisions=False,
    validate_with_memory=True,
    use_smote=False,
    **args)

/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


torch.Size([3, 536, 89])
tensor([0.7742, 0.2260, 0.2156, 0.7577], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9384), tensor(0.9863), tensor(0.0068)]
______epoch 0 _____
val reward 1.4415979385375977
imitation reward 2.036529779434204
distributions [tensor(0.9384), tensor(0.9863), tensor(0.0068)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5379807692307692}, {'decision': 1, 'accuracy': 0.2602739726027397, 'auc': 0.49999999999999994}, {'decision': 2, 'accuracy': 0.5068493150684932, 'auc': 0.5525641025641026}]
tensor([0.7741, 0.2259, 0.2156, 0.7576], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9315), tensor(0.9863), tensor(0.0068)]
______epoch 1 _____
val reward 1.4415051937103271
imitation reward 2.0024476051330566
distributions [tensor(0.9315), tensor(0.9863), tensor(0.0068)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5432692307692308}, {'decision': 1, 'accuracy': 0.2808219178082192, 'auc': 0.4994517543859649

tensor([0.7708, 0.2219, 0.2172, 0.7555], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7534), tensor(0.9452), tensor(0.)]
______epoch 16 _____
val reward 1.4391889572143555
imitation reward 1.634202241897583
distributions [tensor(0.7534), tensor(0.9452), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5620192307692308}, {'decision': 1, 'accuracy': 0.5753424657534246, 'auc': 0.5389254385964912}, {'decision': 2, 'accuracy': 0.7397260273972602, 'auc': 0.6278846153846154}]
tensor([0.7706, 0.2217, 0.2173, 0.7554], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7466), tensor(0.9452), tensor(0.)]
______epoch 17 _____
val reward 1.4389978647232056
imitation reward 1.617250680923462
distributions [tensor(0.7466), tensor(0.9452), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5629807692307692}, {'decision': 1, 'accuracy': 0.589041095890411, 'auc': 0.5435855263157895}, {'decision': 2, 'accuracy': 0.746575342

tensor([0.7670, 0.2173, 0.2191, 0.7529], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.5753), tensor(0.8562), tensor(0.)]
______epoch 32 _____
val reward 1.436344861984253
imitation reward 1.446342945098877
distributions [tensor(0.5753), tensor(0.8562), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5889423076923077}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.6006030701754386}, {'decision': 2, 'accuracy': 0.8013698630136986, 'auc': 0.6471153846153846}]
tensor([0.7668, 0.2170, 0.2192, 0.7528], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.5685), tensor(0.8356), tensor(0.)]
______epoch 33 _____
val reward 1.4362430572509766
imitation reward 1.439392328262329
distributions [tensor(0.5685), tensor(0.8356), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5860576923076922}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.6074561403508772}, {'decision': 2, 'accuracy': 0.8013698630

tensor([0.7645, 0.2144, 0.2211, 0.7506], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.4932), tensor(0.6849), tensor(0.)]
______epoch 48 _____
val reward 1.4354397058486938
imitation reward 1.3595370054244995
distributions [tensor(0.4932), tensor(0.6849), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6086538461538462}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6466557017543859}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.6657051282051282}]
tensor([0.7644, 0.2143, 0.2212, 0.7505], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.4932), tensor(0.6781), tensor(0.)]
______epoch 49 _____
val reward 1.4354101419448853
imitation reward 1.3557102680206299
distributions [tensor(0.4932), tensor(0.6781), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6100961538461538}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6466557017543859}, {'decision': 2, 'accuracy': 0.8150684

...

tensor([0.7614, 0.2061, 0.2125, 0.7557], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.6575), tensor(0.0685), tensor(0.)]
______epoch 320 _____
val reward 1.4185885190963745
imitation reward 1.2753007411956787
distributions [tensor(0.6575), tensor(0.0685), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8698630136986302, 'auc': 0.6206730769230769}, {'decision': 1, 'accuracy': 0.7945205479452054, 'auc': 0.6935307017543859}, {'decision': 2, 'accuracy': 0.8424657534246576, 'auc': 0.8256410256410256}]
tensor([0.7614, 0.2061, 0.2125, 0.7557], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.6575), tensor(0.0685), tensor(0.)]
______epoch 321 _____
val reward 1.4185523986816406
imitation reward 1.2756049633026123
distributions [tensor(0.6575), tensor(0.0685), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8698630136986302, 'auc': 0.6211538461538462}, {'decision': 1, 'accuracy': 0.7945205479452054, 'auc': 0.6943530701754386}, {'decision': 2, 'accuracy': 0.8424

tensor([0.7612, 0.2056, 0.2121, 0.7559], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.6986), tensor(0.0616), tensor(0.)]
______epoch 336 _____
val reward 1.4177162647247314
imitation reward 1.290194034576416
distributions [tensor(0.6986), tensor(0.0616), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8698630136986302, 'auc': 0.6211538461538462}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6929824561403509}, {'decision': 2, 'accuracy': 0.8424657534246576, 'auc': 0.8240384615384615}]
tensor([0.7612, 0.2056, 0.2120, 0.7559], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7055), tensor(0.0616), tensor(0.)]
______epoch 337 _____
val reward 1.4176777601242065
imitation reward 1.289975643157959
distributions [tensor(0.7055), tensor(0.0616), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8698630136986302, 'auc': 0.6221153846153846}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6924342105263158}, {'decision': 2, 'accuracy': 0.842465

tensor([0.7606, 0.2048, 0.2117, 0.7559], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7192), tensor(0.0548), tensor(0.)]
______epoch 352 _____
val reward 1.416542887687683
imitation reward 1.3036088943481445
distributions [tensor(0.7192), tensor(0.0548), tensor(0.)]
[{'decision': 0, 'accuracy': 0.863013698630137, 'auc': 0.6307692307692309}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6921600877192983}, {'decision': 2, 'accuracy': 0.8424657534246576, 'auc': 0.8230769230769232}]
tensor([0.7607, 0.2049, 0.2117, 0.7559], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7260), tensor(0.0548), tensor(0.)]
______epoch 353 _____
val reward 1.4165675640106201
imitation reward 1.3042705059051514
distributions [tensor(0.7260), tensor(0.0548), tensor(0.)]
[{'decision': 0, 'accuracy': 0.863013698630137, 'auc': 0.6317307692307693}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6918859649122807}, {'decision': 2, 'accuracy': 0.8424657

tensor([0.7603, 0.2042, 0.2113, 0.7561], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7466), tensor(0.0479), tensor(0.)]
______epoch 368 _____
val reward 1.4154832363128662
imitation reward 1.3223901987075806
distributions [tensor(0.7466), tensor(0.0479), tensor(0.)]
[{'decision': 0, 'accuracy': 0.863013698630137, 'auc': 0.63125}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6913377192982456}, {'decision': 2, 'accuracy': 0.8424657534246576, 'auc': 0.8221153846153846}]
tensor([0.7604, 0.2043, 0.2112, 0.7561], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7397), tensor(0.0479), tensor(0.)]
______epoch 369 _____
val reward 1.4155620336532593
imitation reward 1.3223991394042969
distributions [tensor(0.7397), tensor(0.0479), tensor(0.)]
[{'decision': 0, 'accuracy': 0.863013698630137, 'auc': 0.6307692307692309}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6899671052631579}, {'decision': 2, 'accuracy': 0.8424657534246576,

tensor([0.7601, 0.2037, 0.2109, 0.7562], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7740), tensor(0.0479), tensor(0.)]
______epoch 384 _____
val reward 1.4146679639816284
imitation reward 1.342106819152832
distributions [tensor(0.7740), tensor(0.0479), tensor(0.)]
[{'decision': 0, 'accuracy': 0.863013698630137, 'auc': 0.6298076923076923}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6855811403508771}, {'decision': 2, 'accuracy': 0.8493150684931506, 'auc': 0.821474358974359}]
tensor([0.7600, 0.2036, 0.2109, 0.7562], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7671), tensor(0.0411), tensor(0.)]
______epoch 385 _____
val reward 1.4145554304122925
imitation reward 1.3441461324691772
distributions [tensor(0.7671), tensor(0.0411), tensor(0.)]
[{'decision': 0, 'accuracy': 0.863013698630137, 'auc': 0.6307692307692307}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.685032894736842}, {'decision': 2, 'accuracy': 0.849315068

In [ ]:
decision_model, _, _, _ = train_decision_model(
    emodel1,emodel2,emodel3,
    lr=.0001,
    use_attention=True,
    imitation_weight=0.1,
    reward_weight=1,
    weights=[1,1,0,0],
    threshold_decisions=False,
    validate_with_memory=True,
    use_smote=True,
    **args)

In [ ]:
decision_model, _, _, _ = train_decision_model(
    emodel1,emodel2,emodel3,
    lr=.0001,
    use_attention=True,
    imitation_weight=0.1,
    reward_weight=1,
    weights=[1,1,1,0],
    threshold_decisions=False,
    validate_with_memory=True,
    use_smote=False,
    **args)

In [ ]:
decision_model, _, _, _ = train_decision_model(
    emodel1,emodel2,emodel3,
    lr=.0001,
    use_attention=True,
    imitation_weight=0.1,
    reward_weight=1,
    weights=[1,1,1,1],
    threshold_decisions=False,
    validate_with_memory=True,
    use_smote=False,
    **args)

In [237]:
def gridsearch_decision_model(m1,m2,m3,weights=[1,1,0,0]):
    model_arglist = [

        {
            'hidden_layers': [100,100],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [50,50],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [2,2],
        },
        {
            'hidden_layers': [300],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [600],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [300,300],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [600,600],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [1000,100],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [1000,100],
            'attention_heads': [5],
        },
    ]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    best_record = {}
    k = 0
    records = []
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for embed_size in [0,120]:
            #embed_size = 0 skips the firt layer that makes the sizes right
            if embed_size == 0 and args['attention_heads'][0] != 1:
                continue
            args['embed_size'] = embed_size
            for dropout in [.5,.75]:
                args['dropout'] = dropout
                for input_dropout in [0,.5]:
                    args['input_dropout'] = input_dropout
                    for shufflecol_chance in [.5]:
                        for use_smote in [True,False]:
                            try:
                                args['shufflecol_chance'] = shufflecol_chance
                                model,m_metrics,m_loss,m_distribution = train_decision_model(m1,m2,m3,
                                                                              use_attention=True,
                                                                              validate_with_memory = True,
                                                                            weights=weights,
                                                                            use_smote=use_smote,
                                                                              verbose=False,
                                                                                             **args)
                                entry = {
                                    'metrics': m_metrics,
                                    'loss': m_loss,
                                    'args': args,
                                    'distribution': m_distribution,
                                    'smote': use_smote
                                }
                                records.append(entry)
                                print('done',k,m_loss)
                                print('curr best',best_loss)
                                k+=1
                                if m_loss < best_loss:
                                    best_loss = m_loss
                                    best_metrics  = m_metrics
                                    best_model = model
                                    best_args = args
                                    best_record = entry
                                    print('_++++++++++New Best++++____')
                                    print(best_loss)
                                    print(best_metrics)
                                    print(best_args)
                                    print('___________')
                                    print('++++++++')
                                    print()
                            except Exception as e:
                                print('error',e,args)
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model, records, best_record

# from Models import *
# decision_model,records, bestr = gridsearch_decision_model(model1,model2,model3_smote,weights=[1,1,.1,0])
# records

/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


here
torch.Size([3, 536, 89])


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


++++++++++Final+++++++++++
best tensor(-0.5596, grad_fn=<SubBackward0>)
[{'decision': 0, 'accuracy': 0.7876712328767124, 'auc': 0.6675480769230769}, {'decision': 1, 'accuracy': 0.2602739726027397, 'auc': 0.48629385964912275}, {'decision': 2, 'accuracy': 0.18493150684931506, 'auc': 0.4080128205128205}]
done 0 tensor(-0.5596, grad_fn=<SubBackward0>)
curr best 100000000000
_++++++++++New Best++++____
tensor(-0.5596, grad_fn=<SubBackward0>)
[{'decision': 0, 'accuracy': 0.7876712328767124, 'auc': 0.6675480769230769}, {'decision': 1, 'accuracy': 0.2602739726027397, 'auc': 0.48629385964912275}, {'decision': 2, 'accuracy': 0.18493150684931506, 'auc': 0.4080128205128205}]
{'hidden_layers': [100, 100], 'attention_heads': [1, 1], 'embed_size': 0, 'dropout': 0.5, 'input_dropout': 0, 'shufflecol_chance': 0.5}
___________
++++++++



/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


...

/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


torch.Size([3, 536, 89])
error mat1 and mat2 shapes cannot be multiplied (2430x1000 and 100x6) {'hidden_layers': [1000, 100], 'attention_heads': [5], 'embed_size': 120, 'dropout': 0.75, 'input_dropout': 0.5, 'shufflecol_chance': 0.5}


/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:249: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


torch.Size([3, 536, 89])
error mat1 and mat2 shapes cannot be multiplied (390x1000 and 100x6) {'hidden_layers': [1000, 100], 'attention_heads': [5], 'embed_size': 120, 'dropout': 0.75, 'input_dropout': 0.5, 'shufflecol_chance': 0.5}
_________
+++++++++++
best stuff tensor(-1.0320, grad_fn=<SubBackward0>)
[{'decision': 0, 'accuracy': 0.5958904109589042, 'auc': 0.6951923076923077}, {'decision': 1, 'accuracy': 0.2191780821917808, 'auc': 0.6038925438596492}, {'decision': 2, 'accuracy': 0.8356164383561644, 'auc': 0.7352564102564102}]
{'hidden_layers': [50, 50], 'attention_heads': [1, 1], 'embed_size': 120, 'dropout': 0.75, 'input_dropout': 0.5, 'shufflecol_chance': 0.5}


[{'metrics': [{'decision': 0,
    'accuracy': 0.7876712328767124,
    'auc': 0.6675480769230769},
   {'decision': 1, 'accuracy': 0.2602739726027397, 'auc': 0.48629385964912275},
   {'decision': 2,
    'accuracy': 0.18493150684931506,
    'auc': 0.4080128205128205}],
  'loss': tensor(-0.5596, grad_fn=<SubBackward0>),
  'args': {'hidden_layers': [100, 100],
   'attention_heads': [1, 1],
   'embed_size': 120,
   'dropout': 0.75,
   'input_dropout': 0.5,
   'shufflecol_chance': 0.5},
  'distribution': [tensor(1.), tensor(0.), tensor(0.)],
  'smote': True},
 {'metrics': [{'decision': 0,
    'accuracy': 0.1095890410958904,
    'auc': 0.5889423076923077},
   {'decision': 1, 'accuracy': 0.4315068493150685, 'auc': 0.45559210526315785},
   {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.7141025641025641}],
  'loss': tensor(-0.7563, grad_fn=<SubBackward0>),
  'args': {'hidden_layers': [100, 100],
   'attention_heads': [1, 1],
   'embed_size': 120,
   'dropout': 0.75,
   'input_dropout': 0

In [243]:
decision_model

DecisionAttentionModel(
  (input_dropout): Dropout(p=0, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=120, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
  )
  (batchnorm): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): Softplus(beta=1, threshold=20)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (final_layer): Linear(in_features=50, out_features=6, bias=True)
  (resize_layer): Linear(in_features=93, out_features=120, bias=True)
  (attentions): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=120, out_features=120, bias=True)
    )
    (1): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=50, out_features=50, bias=True)
    )
  )
  (norms): ModuleList(
    (0): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
    (1): LayerNorm((50,), eps=1e

In [45]:
torch.save(decision_model,'../data/models/final_decision_model_' + decision_model.identifier + '.pt')
print('../data/models/final_decision_model_' + decision_model.identifier + '.pt')

../data/models/final_decision_model_statedecisions_input126_dims50,50_dropout0,0.5.pt


In [147]:
# torch.save(model1,'../data/models/final_transition1_model_' + emodel1.identifier + '.pt')
# torch.save(model2,'../data/models/final_transition2_model_' + emodel2.identifier + '.pt')
# torch.save(model3,'../data/models/final_outcome_model_' + emodel3.identifier + '.pt')
# print('../data/models/final_transition1_model_' + emodel1.identifier + '.pt')
# print('../data/models/final_transition2_model_' + emodel2.identifier + '.pt')
# print('../data/models/final_outcome_model_' + emodel3.identifier + '.pt')

AttributeError: 'TransitionEnsemble' object has no attribute 'identifier'

In [47]:
from Models import *
decision_model.set_device('cpu')
emodel1.set_device('cpu')
emodel2.set_device('cpu')
emodel3.set_device('cpu')
torch.save(decision_model,'../resources/decision_model.pt')
torch.save(emodel1,'../resources/transition1_model.pt')
torch.save(emodel2,'../resources/transition2_model.pt')
torch.save(emodel3,'../resources/outcome_model.pt')

In [ ]:
def get_default_predictions(dm):
    res  = []
    for state in [0,1,2]:
        mem = dm.memory[state]
        mem = torch.median(mem,dim=0)[0].type(torch.FloatTensor)
        val = dm(mem.reshape(1,-1),position=state)
        res.append(val.cpu().detach().numpy())
    return np.stack(res)
get_default_predictions(decision_model)